In [ ]:
# ✅ Step 1: Install dependencies
!pip install albumentations opencv-python

In [ ]:
# ✅ Step 2: Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# ✅ Step 3: Imports
import os
import cv2
import albumentations as A
import shutil

In [ ]:
# ✅ Step 4: Paths
base_drive_path = "/content/drive/MyDrive/Lõputöö Mudel"
input_img_folder = os.path.join(base_drive_path, "data/images")
input_lbl_folder = os.path.join(base_drive_path, "data/labels")

output_img_folder = os.path.join(base_drive_path, "aug/images")
output_lbl_folder = os.path.join(base_drive_path, "aug/labels")
os.makedirs(output_img_folder, exist_ok=True)
os.makedirs(output_lbl_folder, exist_ok=True)

In [ ]:
# ✅ Step 5: Albumentations augmentation pipeline

transform = A.Compose([
    # Simulate camera drift or slight tilt as the drone moves
    A.ShiftScaleRotate(
        shift_limit=0.05,     # Small shifts, max ±5% of width/height
        scale_limit=0.05,     # Tiny scale changes
        rotate_limit=5,       # Slight rotations only (to simulate small tilts)
        p=0.7
    ),

    # Horizontal flip is okay; vertical is usually unrealistic unless drone flips (unlikely)
    A.HorizontalFlip(p=0.5),

    # Simulate slight environmental noise from heat wave distortions or movement blur
    A.MotionBlur(blur_limit=3, p=0.2),

    # Rare distortions or image noise to simulate sensor artifacts
    A.GaussNoise(var_limit=(5.0, 20.0), p=0.2),

    # Slight brightness/contrast changes (thermal cameras can vary per frame)
    A.RandomBrightnessContrast(
        brightness_limit=0.05,
        contrast_limit=0.05,
        p=0.3
    )
], bbox_params=A.BboxParams(format='yolo', label_fields=['class_labels']))


/usr/local/lib/python3.11/dist-packages/albumentations/core/validation.py:111: UserWarning: ShiftScaleRotate is a special case of Affine transform. Please use Affine transform instead.
  original_init(self, **validated_kwargs)
<ipython-input-5-24bcefe36177>:19: UserWarning: Argument(s) 'var_limit' are not valid for transform GaussNoise
  A.GaussNoise(var_limit=(5.0, 20.0), p=0.2),


In [ ]:
# ✅ Step 6: Helper functions
def load_yolo_labels(label_path):
    boxes = []
    class_labels = []
    with open(label_path, 'r') as f:
        for line in f:
            parts = line.strip().split()
            if len(parts) != 5:
                continue
            class_id = int(parts[0])
            bbox = list(map(float, parts[1:]))
            boxes.append(bbox)
            class_labels.append(class_id)
    return boxes, class_labels

def save_yolo_labels(label_path, boxes, class_labels):
    with open(label_path, 'w') as f:
        for bbox, cls in zip(boxes, class_labels):
            f.write(f"{cls} {' '.join(f'{x:.6f}' for x in bbox)}\n")


In [ ]:
# ✅ Step 7: Augment and save images + labels
for file in os.listdir(input_img_folder):
    if not file.endswith(".jpg"):
        continue

    img_path = os.path.join(input_img_folder, file)
    label_path = os.path.join(input_lbl_folder, file.replace(".jpg", ".txt"))

    image = cv2.imread(img_path)
    if image is None:
        continue

    boxes, class_labels = load_yolo_labels(label_path)
    if not boxes:
        continue

    # Apply augmentation
    transformed = transform(image=image, bboxes=boxes, class_labels=class_labels)
    aug_img = transformed['image']
    aug_boxes = transformed['bboxes']
    aug_labels = transformed['class_labels']

    # Save augmented image and label
    aug_img_path = os.path.join(output_img_folder, f"aug_{file}")
    aug_lbl_path = os.path.join(output_lbl_folder, f"aug_{file.replace('.jpg', '.txt')}")

    cv2.imwrite(aug_img_path, aug_img)
    save_yolo_labels(aug_lbl_path, aug_boxes, aug_labels)

print("✅ Augmented images created.")

✅ Augmented images created.


In [ ]:
# ✅ Step 8: Copy original data to the same output folders
for file in os.listdir(input_img_folder):
    if file.endswith(".jpg"):
        shutil.copy2(os.path.join(input_img_folder, file),
                     os.path.join(output_img_folder, file))

for file in os.listdir(input_lbl_folder):
    if file.endswith(".txt"):
        shutil.copy2(os.path.join(input_lbl_folder, file),
                     os.path.join(output_lbl_folder, file))

print("✅ Original images and labels copied.")
print("✅ Final dataset is ready in Google Drive.")

✅ Original images and labels copied.
✅ Final dataset is ready in Google Drive.


In [ ]:
# prompt: get how many images in the aug folder

import os

aug_image_folder = "/content/drive/MyDrive/Lõputöö Mudel/aug/images"
image_count = len([f for f in os.listdir(aug_image_folder) if os.path.isfile(os.path.join(aug_image_folder, f))])
print(f"Number of images in the 'aug/images' folder: {image_count}")


Number of images in the 'aug/images' folder: 11451
